# Access Neighbors
  
In this notebook, we provides neighboring traveling examples using Trimesh.

In [1]:
import sys,os

RES_PATH = '../../../../resources'
LIB_PATH = '../../../../python_lib'

if not os.path.exists(RES_PATH):
    print( 'cannot find \resources\, please update RES_PATH')
    exit(1)
    

# append path 
sys.path.append(LIB_PATH) 
from geo_tools import rd_helper

import pyglet
pyglet.options['shadow_window'] = False

import pyrender
import numpy as np
import trimesh

import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
trimesh.__version__

'3.9.1'

### some handy functions for display

In [3]:
def scene_factory(render_list):
    
    scene = pyrender.Scene(ambient_light=0.5*np.array([1.0, 1.0, 1.0, 1.0]))
    
    for m in render_list:
        scene.add(m)
    
    return scene

def run_gui(render_list, **kargs ):
    scene = scene_factory(render_list)
    # call GUI
    v=pyrender.Viewer(scene, use_raymond_lighting=True, **kargs)
    # free resources
    del v 
    return None


## Load a mesh

In [4]:
mesh_fp = os.path.join(RES_PATH,'torus.obj')
assert os.path.exists(mesh_fp), 'cannot found:'+mesh_fp 
tm = trimesh.load(mesh_fp) 


## Access Vertices, Faces, Edges

In [5]:
props= ['vertices', 'faces', 'edges', 'face_normals' , 'vertex_normals']

for p in props:
    print('{:}, shape={:}, #={:}'.format(p,str(getattr(tm,p).shape),getattr(tm,p).shape[0]))

vertices, shape=(288, 3), #=288
faces, shape=(576, 3), #=576
edges, shape=(1728, 2), #=1728
face_normals, shape=(576, 3), #=576
vertex_normals, shape=(288, 3), #=288


In [6]:
print('vertices[0]=',tm.vertices[0,:])
print('faces[0]=',tm.faces[0,:])
print('edges[0]=',tm.edges[0,:])

vertices[0]= [ 0. -2. -0.]
faces[0]= [203  41  32]
edges[0]= [203  41]


## Apply a transform

In [20]:
print('vertices[0]=',tm.vertices[0,:])

tx = np.eye(4)
tx[:3,3]=np.array((0,10,0))

# update vertices by matrix transformation
print('4-by-4 matrix=')
print(tx)
tm2=tm.copy()
tm2.apply_transform(tx)

print('vertices[0]=',tm2.vertices[0,:])

# directly update vertices
tm3=tm.copy()
tm3.vertices+= np.array([0,10,10]).reshape(1,3)

print('vertices[0]=',tm3.vertices[0,:])

vertices[0]= [ 0. -2. -0.]
4-by-4 matrix=
[[ 1.  0.  0.  0.]
 [ 0.  1.  0. 10.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
vertices[0]= [0. 8. 0.]
vertices[0]= [ 0.  8. 10.]


In [19]:
mesh_rd1 = pyrender.Mesh.from_trimesh(tm)
mesh_rd2 = pyrender.Mesh.from_trimesh(tm2)
mesh_rd3 = pyrender.Mesh.from_trimesh(tm3)

run_gui([mesh_rd1,mesh_rd2,mesh_rd3])

## Select some edges via Trimesh.edges

<img src="../imgs/s_edges2.jpg" width="350" /> 


In [24]:
edges = tm.edges[:30:4,:]

p0 = tm.vertices[edges[:,0],:]
p1 = tm.vertices[edges[:,1],:]

edge_obj = rd_helper.create_edges(p0, p1, line_color=(1.0,0.0,0.0))

mesh_rd1 = pyrender.Mesh.from_trimesh(tm)

run_gui(render_list=[edge_obj,mesh_rd1],point_size=10.0)

## Select some faces via Trimesh.faces

<img src="../imgs/s_faces3.jpg" width="350" /> 


In [26]:
faces = tm.faces[:10,:]

# a handy function to create face visualization
def cook_face_vismesh(tm_obj, sel_faces):
    p0 = tm_obj.vertices[sel_faces[:,0],:]
    p1 = tm_obj.vertices[sel_faces[:,1],:]
    p2 = tm_obj.vertices[sel_faces[:,2],:]

    edge_obj_list=[]

    c=(1.0,0.0,0.0)
    edge_obj_list.append( rd_helper.create_edges(p0, p1, p0_color=c,p1_color=c,line_color=c , no_vex=True) )
    edge_obj_list.append( rd_helper.create_edges(p1, p2, p0_color=c,p1_color=c,line_color=c , no_vex=True) )
    edge_obj_list.append( rd_helper.create_edges(p2, p0, p0_color=c,p1_color=c,line_color=c , no_vex=True) )
    
    return edge_obj_list


rd_list = cook_face_vismesh(tm, sel_faces=faces)

mesh_rd1 = pyrender.Mesh.from_trimesh(tm)

rd_list.append(mesh_rd1)

run_gui( rd_list , point_size=10.0)

## Select neighboring vertices (red colored)
<img src="../imgs/s_cb.jpg" width="400" /> 


### Load a box mesh

In [27]:
mesh_fp = os.path.join(RES_PATH,'box.obj')
assert os.path.exists(mesh_fp), 'cannot found:'+mesh_fp 
box_tm = trimesh.load(mesh_fp) 


In [12]:
def test_nbv():
    vex_index = 5
    
    vex = box_tm.vertices[vex_index,:].reshape(1,3)
    
    #https://trimsh.org/trimesh.html#trimesh.Trimesh.vertex_neighbors
    nb_vindices =box_tm.vertex_neighbors[vex_index]
    
    print('vertex[{:}]'.format(vex))
    print('vertex_neighbors:',nb_vindices)
    
    nb_vxyz = box_tm.vertices[nb_vindices,:]
    
    # add a vertex into the rendering list
    render_list=[]
    
    # vextex must be (N,3)
    render_list.append( pyrender.Mesh.from_points( vex, colors=(0.0,1.0,0.0))  )
    render_list.append( pyrender.Mesh.from_points( nb_vxyz, colors=(1.0,0.0,0.0))   )
    
    # also add the box mesh
    render_list.append( pyrender.Mesh.from_trimesh( box_tm)   )
    
    # call GUI
    run_gui(render_list=render_list, point_size=20.0, all_wireframe=True, cull_faces=True)
    

test_nbv()

vertex[[[ 0.5 -0.5  0.5]]]
vertex_neighbors: [1, 3, 4, 6, 7]


## Select neighboring faces (red colored)

<img src="../imgs/nb_face2.png" width="400" /> 


In [13]:
mesh_fp = os.path.join(RES_PATH,'hfsphere.obj')
assert os.path.exists(mesh_fp), 'cannot found:'+mesh_fp 
hfsphere_tm = trimesh.load(mesh_fp) 

In [14]:

def test_nbf():
    
    tm_obj = hfsphere_tm
    
    vex_index = 4
    vex = tm_obj.vertices[vex_index,:].reshape(1,3)
    
    # get its neighboring faces
    face_indices = tm_obj.vertex_faces[vex_index]

    # select faces
    faces = tm_obj.faces[face_indices,:]
    
    #-----------------------------------------------------
    # create visualization object
    
    render_list=[]
    
    # add faces
    render_list = render_list + cook_face_vismesh(tm_obj=tm_obj, sel_faces=faces)
    
    # add a vertex into the rendering list    
    pts_m = pyrender.Mesh.from_points( vex, colors=(0.0,1.0,0.0)) 
    render_list.append(pts_m)
    
    # also add mesh 
    mesh_obj = pyrender.Mesh.from_trimesh(tm_obj)
    render_list.append(mesh_obj)

    # call GUI
    run_gui(render_list=render_list, point_size=10.0, all_wireframe=False, cull_faces=True)

    
test_nbf()